LeNet、AlexNet和VGG都有一个共同的设计模式：
通过一系列的卷积层与汇聚层来提取空间结构特征；
然后通过全连接层对特征的表征进行处理。
AlexNet和VGG对LeNet的改进主要在于如何扩大和加深这两个模块。

##### 如果使用了全连接层，可能会完全放弃表征的空间结构
网络中的网络（NiN）提供了一个非常简单的解决方案：
##### 在每个像素的通道上，分别使用MLP

### 7.3.1. NiN块
回想一下，卷积层的输入和输出由四维张量组成，张量的每个轴分别对应样本、通道、高度和宽度。
##### B C W H
另外，全连接层的输入和输出通常分别对应样本和特征的二维张量

NiN的想法是，在像素的每个位置（针对W和H）应用一个全连接层。
如果把权重连接到每个空间位置，我们可以将其视为1*1卷积，
或在每个像素位置上独立作用的全连接层。
从另一个角度看，即空间维度的每个像素视为单个样本，把通道维度视为不同特征。
![vgg_nin](imgs/7_3_1vgg_nin1.png)

+ NiN块，以一个普通卷积层开始，
+ 后面是两个1 * 1卷积。 这两个 1 * 1卷积充当带有ReLU激活函数的逐像素全连接层。
+ 第一层的卷积窗口形状通常由用户设置。
+ 随后的卷积窗口形状固定为 1×1 。

In [1]:
import torch
from torch import nn
from d2l import torch as d2l

### 7.3.2. NiN模型
NiN使用窗口形状为11 x 11、 5 x 5、 3 x 3的卷积层，输出通道数量与AlexNet中相同。
每个NiN块后有一个最大池化，size为3 x 3， strider为2。

NiN 和 AlexNet之间的一个显著区别是
##### NiN完全取消了全连接层。
相反，NiN使用一个NiN块，其输出通道数等于类别的数量。
最后放一个全局平均池化（global average pooling layer）
生成一个多元逻辑向量（logits）。

NiN显著增加了模型所需要的参数数量，但是在实践中，这种设计常常会增加训练时间。

In [16]:
def nin_block(in_channels, out_channels, kernel_size, strides, padding):
    return nn.Sequential(
        nn.Conv2d(in_channels, out_channels, kernel_size, strides, padding),
        nn.ReLU(),
        nn.Conv2d(out_channels, out_channels, kernel_size=1), nn.ReLU(),
        nn.Conv2d(out_channels, out_channels, kernel_size=1), nn.ReLU())

In [17]:
net = nn.Sequential(
    nin_block(1, 96, kernel_size=11, strides=4, padding=0),
    nn.MaxPool2d(3, stride=2),
    nin_block(96, 256, kernel_size=5, strides=1, padding=2),
    nn.MaxPool2d(3, stride=2),
    nin_block(256, 384, kernel_size=3, strides=1, padding=1),
    nn.MaxPool2d(3, stride=2),
    nn.Dropout(0.5),
    # 标签类别数是10
    nin_block(384, 10, kernel_size=3, strides=1, padding=1),
    nn.AdaptiveAvgPool2d((1, 1)),
    # 将四维的输出转成二维的输出，其形状为(批量大小,10)
    nn.Flatten())

In [18]:
X = torch.rand(size=(1, 1, 224, 224))
for layer in net:
    X = layer(X)
    print(layer.__class__.__name__,'output shape:\t', X.shape)

Sequential output shape:	 torch.Size([1, 96, 54, 54])
MaxPool2d output shape:	 torch.Size([1, 96, 26, 26])
Sequential output shape:	 torch.Size([1, 256, 26, 26])
MaxPool2d output shape:	 torch.Size([1, 256, 12, 12])
Sequential output shape:	 torch.Size([1, 384, 12, 12])
MaxPool2d output shape:	 torch.Size([1, 384, 5, 5])
Dropout output shape:	 torch.Size([1, 384, 5, 5])
Sequential output shape:	 torch.Size([1, 10, 5, 5])
AdaptiveAvgPool2d output shape:	 torch.Size([1, 10, 1, 1])
Flatten output shape:	 torch.Size([1, 10])


In [19]:
lr, num_epochs, batch_size = 0.1, 10, 128
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size, resize=224)
d2l.train_ch6(net, train_iter, test_iter, num_epochs, lr, d2l.try_gpu())

TypeError: 'int' object is not callable